In [5]:
# Cell 1: Import Required Libraries
# =====================================

# Core Python libraries
import json
import os
from typing import Dict, List, Any, Optional, Tuple
from datetime import datetime
import base64
from io import BytesIO

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns

# HTTP requests for API calls
import requests

# Environment variables (for API keys)
from dotenv import load_dotenv

# Rich text formatting for better notebook output
from IPython.display import display, HTML, Image, Markdown

# Warnings management
import warnings
warnings.filterwarnings('ignore')

# Set visualization defaults
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load environment variables if .env file exists
load_dotenv()

False

In [8]:
# Cell 2: Configuration and API Setup
# =====================================

# API Configuration
OPENROUTER_API_KEY = "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"
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL_NAME = "deepseek/deepseek-chat-v3-0324:free"  # Deepseek model via OpenRouter

# Headers for API requests
HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "http://localhost:8888",  # Required for OpenRouter
    "X-Title": "Dental Practice Valuation Q&A"  # Optional, helps with OpenRouter analytics
}

# Question Categories (for routing to appropriate handlers)
QUESTION_CATEGORIES = {
    "P&L_ANALYSIS": ["profit", "revenue", "expense", "margin", "ebitda", "income"],
    "TAX": ["tax", "deduction", "write-off", "depreciation", "amortization"],
    "VALUATION": ["multiple", "valuation", "worth", "value", "price"],
    "OPERATIONS": ["patient", "operatories", "technology", "location", "service"],
    "SDE": ["sde", "seller", "discretionary", "earnings", "addback", "adjustment"],
    "SWOT": ["strength", "weakness", "opportunity", "threat", "swot", "growth"],
    "DEBT": ["debt", "loan", "financing", "interest", "service"],
    "GENERAL": ["tell me about", "overview", "summary", "explain"]
}

# Visualization preferences
VIZ_CONFIG = {
    "figure_size": (10, 6),
    "color_scheme": ["#2E86AB", "#A23B72", "#F18F01", "#C73E1D", "#6A994E"],
    "font_size": 12,
    "title_size": 14,
    "dpi": 100,
    "style": "seaborn-v0_8-darkgrid"
}

# Financial formatting preferences
FINANCIAL_CONFIG = {
    "currency_symbol": "$",
    "thousands_separator": ",",
    "decimal_places": 0,
    "percentage_decimal": 1
}

# System prompt for the LLM
SYSTEM_PROMPT = """You are a dental practice valuation expert AI assistant. You analyze financial data, 
operational metrics, and practice characteristics to provide insights about dental practice valuations.

Your responses should be:
1. Accurate and based solely on the provided data
2. Professional but accessible to non-financial professionals
3. Focused on actionable insights
4. Clear about calculations and methodologies used

When analyzing data:
- Always show key calculations
- Explain financial terms when first used
- Highlight important ratios and benchmarks
- Provide context for valuation multiples
"""

# Test API connection
def test_api_connection():
    """Test the OpenRouter API connection with Deepseek"""
    test_payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Respond with 'Connection successful!' if you receive this."}
        ],
        "max_tokens": 50,
        "temperature": 0.1
    }
    
    try:
        response = requests.post(
            OPENROUTER_BASE_URL,
            headers=HEADERS,
            json=test_payload
        )
        
        if response.status_code == 200:
            result = response.json()
            message = result['choices'][0]['message']['content']
            print(f"✅ API Connection Test Successful!")
            print(f"📡 Model: {MODEL_NAME}")
            print(f"💬 Response: {message}")
            return True
        else:
            print(f"❌ API Connection Failed!")
            print(f"Status Code: {response.status_code}")
            print(f"Error: {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ Connection Error: {str(e)}")
        return False

# Run connection test
print("=" * 50)
print("DENTAL PRACTICE VALUATION Q&A SYSTEM")
print("=" * 50)
print(f"\n🔧 Configuration loaded successfully!")
print(f"🤖 Using model: {MODEL_NAME}")
print(f"\n📡 Testing API connection...")
print("-" * 50)

connection_status = test_api_connection()

if connection_status:
    print("-" * 50)
    print("✅ System ready for use!")
else:
    print("-" * 50)
    print("⚠️  Please check your API key and connection")

DENTAL PRACTICE VALUATION Q&A SYSTEM

🔧 Configuration loaded successfully!
🤖 Using model: deepseek/deepseek-chat-v3-0324:free

📡 Testing API connection...
--------------------------------------------------
✅ API Connection Test Successful!
📡 Model: deepseek/deepseek-chat-v3-0324:free
💬 Response: Connection successful!
--------------------------------------------------
✅ System ready for use!
